<a href="https://colab.research.google.com/github/atqaamir/PriWorld-Privacy-Policy-Analysis/blob/convolutionNN/PriWorldFYP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
from google.colab import files
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D , Conv2DTranspose#, Conv2DTranspose
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense
from keras.models import Model

import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import os
import csv
import pandas as pd
import io
from skimage import io
import numpy as np
import re

Using TensorFlow backend.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from keras.layers import Conv1D
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [3]:
!pip install gensim 

In [4]:

!ls "/content/drive/My Drive/FYP/Final Year Project Code/5Nov"

customizedWord2Vec.csv	FYPv5.2_letterTrigram.ipynb  PP_sentences.txt
FYPv5.1_Padding.ipynb	FYPv5.ipynb		     withoutlemmaTri.csv


In [0]:
#model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)

In [0]:
documents = open('/content/drive/My Drive/FYP/Final Year Project Code/dataSet.txt')

In [0]:
labels = open('/content/drive/My Drive/FYP/Final Year Project Code/labels.txt')

In [0]:
Xdata = documents.readlines()
Y = labels.readlines()

In [8]:
Xdata[:5],Y[:5]

(['   Block the repeated delivery of the survey either in the current visit or in any subsequent visit    Record only that the visitor had the opportunity to answer the survey questions    Expire  days after being set    Reduce the burden on visitors to the site by avoiding repeated delivery of the popup survey \n',
  '   Collect any information about visitors    Track the web surfing activities of visitors    Indicate whether a visitor answered any questions    Link a particular visitor to any response of set of responses to the survey \n',
  '   Conduct market research and audience analysis    For any other purpose described at the point of collection or with your consent\n',
  '   Cookies are unique identifiers that we transfer to your device to enable our systems to recognize your device and to provide features such as Click purchasing Recommended for You  personalized advertisements on other Web sites e.g\n',
  '   Device informationsuch as your hardware model IP address other uni

In [0]:
# file.close()

In [10]:
cleanX = []
cleanY = []
for i in range(len(Xdata)):
  cleanX.append(Xdata[i].replace('\n','').replace('[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]',''))
  cleanY.append(Y[i].replace('\n',''))
cleanX[:5],cleanY[:5]

(['   Block the repeated delivery of the survey either in the current visit or in any subsequent visit    Record only that the visitor had the opportunity to answer the survey questions    Expire  days after being set    Reduce the burden on visitors to the site by avoiding repeated delivery of the popup survey ',
  '   Collect any information about visitors    Track the web surfing activities of visitors    Indicate whether a visitor answered any questions    Link a particular visitor to any response of set of responses to the survey ',
  '   Conduct market research and audience analysis    For any other purpose described at the point of collection or with your consent',
  '   Cookies are unique identifiers that we transfer to your device to enable our systems to recognize your device and to provide features such as Click purchasing Recommended for You  personalized advertisements on other Web sites e.g',
  '   Device informationsuch as your hardware model IP address other unique devi

In [0]:
def parse_string(string): 
#     string = string.replace('_','')
#     string = string.replace('-','')
#     string = re.sub("[^a-zA-Z]",' ',string)
    string = ' '.join(string.split())
    token_list = list(string.split())
    
    return token_list


In [0]:
text_list=[]
X=np.array(cleanX)

# print len(X)
# print X[1]
for iterator in range(0,len(X)):
    text_list.append(parse_string(str(X[iterator])))

In [0]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import nltk
import re
import bs4
import gensim.downloader as api

In [15]:
model = Word2Vec(text_list, min_count=1, compute_loss=True, size=100)
# # summarize the loaded model
print(model)

Word2Vec(vocab=5215, size=100, alpha=0.025)


In [0]:
model = Word2Vec.load('/content/drive/My Drive/FYP/Final Year Project Code/PP_word2vec_new.bin')

In [17]:
for count in range(0,2):
    model.train(text_list,total_examples=len(text_list), epochs=500, compute_loss=True,start_alpha=0.1)
    print (model.get_latest_training_loss())

94172.8046875
90685.1015625


In [0]:
# save model
model.save('/content/drive/My Drive/FYP/Final Year Project Code/PP_word2vec_new.bin')
# # load model


In [0]:

def s2V(s):
  
  sent = np.zeros((100,))
  
  for w in s.split() :
#     #print (np.array(df[Lemmatization(w)]).shape)
#     print (sent)
#     print (model[w])
    sent += np.array(model[w])
  
  return np.array(sent)



In [28]:

secVectors = []
cleanX[0]
for sen in cleanX:
  secVectors.append(s2V(sen))


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [29]:
np.unique(np.array(cleanY))

array(['data retention', 'data security', 'do not track',
       'first party collection/use',
       'international and specific audiences', 'other', 'policy change',
       'third party sharing/collection', 'user access, edit and deletion',
       'user choice/control'], dtype='|S36')

In [30]:

print (np.shape(cleanX))
print (np.shape(secVectors))
  

(4734,)
(4734, 100)


In [31]:
trainsize = int(np.array(secVectors).shape[0]*0.75)
trainsize

3550

In [0]:
secVectors = np.array(secVectors).reshape((4734, 100, 1))
s = np.arange((secVectors).shape[0])
np.random.shuffle(s)
secVectors = secVectors[s]

In [0]:

# define model

model = Sequential()

model.add(Conv1D( 200,3, activation='relu',input_shape=np.shape(secVectors)[1:],kernel_initializer='glorot_uniform') )

model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(150,3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(100,3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
          
model.add(Dense(50, activation = "relu"))
model.add(Flatten())
model.add(Dense(10, activation = "sigmoid"))


In [35]:
sgd = keras.optimizers.SGD(lr = 0.05)
adam = keras.optimizers.Adam(lr=0.1)
model.compile(optimizer=adam, loss='mean_squared_error', metrics=['acc'])
print (model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 98, 200)           800       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 49, 200)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 47, 150)           90150     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 23, 150)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 21, 100)           45100     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 10, 100)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 10, 50)            5050      
__________

In [0]:
cleanY = np.array(cleanY)
cleanY = cleanY[s]
import pandas as pd
s = pd.Series(np.unique(cleanY))

tags = (pd.get_dummies(s)).columns.values
onehot = np.array(pd.get_dummies(s))

In [0]:
# u = np.unique(cleanY)
labls = np.zeros((cleanY.shape[0],10))
for i in range(len(tags)):
  labls[cleanY == tags[i]] = onehot[i]

In [38]:
cleanY[:5],labls[:5]

(array(['first party collection/use', 'first party collection/use',
        'first party collection/use', 'user access, edit and deletion',
        'other'], dtype='|S36'),
 array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]]))

In [0]:
# from sklearn.utils import shuffle
# secVectors, labls = shuffle(secVectors, labls, random_state=0)

In [39]:
model.fit(x = np.copy(secVectors)[:trainsize],y= np.copy(labls)[:trainsize], batch_size=200, epochs=10,shuffle=True)

Epoch 1/10
3550/3550 [==============================] - 3s 888us/step - loss: 0.1038 - acc: 0.5572
Epoch 2/10
3550/3550 [==============================] - 0s 105us/step - loss: 0.0828 - acc: 0.5862
Epoch 3/10
3550/3550 [==============================] - 0s 104us/step - loss: 0.0828 - acc: 0.5862
Epoch 4/10
3550/3550 [==============================] - 0s 104us/step - loss: 0.0828 - acc: 0.5862
Epoch 5/10
3550/3550 [==============================] - 0s 104us/step - loss: 0.0828 - acc: 0.5862
Epoch 6/10
3550/3550 [==============================] - 0s 103us/step - loss: 0.0828 - acc: 0.5862
Epoch 7/10
3550/3550 [==============================] - 0s 103us/step - loss: 0.0828 - acc: 0.5862
Epoch 8/10
3550/3550 [==============================] - 0s 104us/step - loss: 0.0828 - acc: 0.5862
Epoch 9/10
3550/3550 [==============================] - 0s 104us/step - loss: 0.0828 - acc: 0.5862
Epoch 10/10
3550/3550 [==============================] - 0s 104us/step - loss: 0.0828 - acc: 0.5862


In [40]:

p=model.predict(np.copy(secVectors)[trainsize:])
print (p)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [41]:
p[13]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [0]:
theek = 0
for i in range(len(p)):
  if np.all(np.array(p[i]) ==np.copy(labls)[trainsize+i]):
    theek +=1


In [44]:
(theek)/float(np.shape(cleanY)[0] - trainsize) 

0.612331081081081